Basic work on getting a HuggingFace model working

In [1]:
!pip install accelerate transformers datasets
!pip install transformers
!pip install matplotlib
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from difflib import SequenceMatcher
from transformers import BigBirdTokenizer, BigBirdForMultipleChoice

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForMultipleChoice.from_pretrained("microsoft/deberta-v3-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
path_to_train = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/train.jsonl"
path_to_dev = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl"
path_to_test = "/content/drive/MyDrive/NLP_FINAL_PROJECT/OpenBookQA-V1-Sep2018/Data/Main/test.jsonl"
data_files = {"train": path_to_train, "validation": path_to_dev, "test": path_to_test}
data = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answerKey'],
        num_rows: 4957
    })
    validation: Dataset({
        features: ['id', 'question', 'answerKey'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'question', 'answerKey'],
        num_rows: 500
    })
})

In [9]:
data['validation'][0]

{'id': '8-376',
 'question': {'stem': 'Frilled sharks and angler fish live far beneath the surface of the ocean, which is why they are known as',
  'choices': [{'text': 'Deep sea animals', 'label': 'A'},
   {'text': 'fish', 'label': 'B'},
   {'text': 'Long Sea Fish', 'label': 'C'},
   {'text': 'Far Sea Animals', 'label': 'D'}]},
 'answerKey': 'A'}

In [7]:
# Mapping from answer keys to numeric values
answer_key_to_numeric = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

# Initialize counters
total_questions = len(data["validation"])

In [12]:
from tqdm import tqdm

In [13]:
def format_inputs_for_multiple_choice(question_stem, choices_text):
    """
    Formats a question and its choices into a model-compatible format,
    by pairing each choice with the question stem before tokenization.
    """
    choices_input = [question_stem + " " + choice for choice in choices_text]  # Combine question stem with each choice
    # Tokenize the inputs for the multiple-choice format. Each choice is tokenized separately but in the context of the question.
    inputs = tokenizer(choices_input, padding=True, return_tensors="pt", truncation=True, max_length=512, add_special_tokens=True)
    return inputs


correct_predictions = 0
total_questions = len(data["validation"])

for item in tqdm(data["validation"], desc="Processing Questions"):
    question_stem = item['question']['stem']
    choices_text = [choice['text'] for choice in item['question']['choices']]

    # Format the inputs
    inputs = format_inputs_for_multiple_choice(question_stem, choices_text)

    # Predict the correct answer

    outputs = model(**{key: value.unsqueeze(0) for key, value in inputs.items()})
    predictions = torch.argmax(outputs.logits, dim=1)
    predicted_choice_index = predictions.item()



    # Map the answer key ('A', 'B', 'C', 'D') to an index (0, 1, 2, 3)
    actual_answer_index = ord(item['answerKey']) - ord('A')

    # Compare the predicted index with the actual answer index
    if predicted_choice_index == actual_answer_index:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_questions
print("Accuracy:", accuracy)

Processing Questions: 100%|██████████| 500/500 [04:34<00:00,  1.82it/s]

Accuracy: 0.156


In [8]:
def match_choice_to_generated_text(generated_text, choices_text):
    # Function to find the choice that best matches the generated text
    best_match = 0
    highest_similarity = 0.0

    for i, choice_text in enumerate(choices_text):
        similarity = SequenceMatcher(None, generated_text, choice_text).ratio()
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = i

    return best_match

correct_predictions = 0
total_questions = len(data["validation"])

for item in data["validation"]:
    question_id = item['id']
    question_stem = item['question']['stem']
    answer_key = item['answerKey']
    choices = item['question']['choices']

    choices_text = [choice['text'] for choice in choices]

    # Since T5 generates text, we format the question for generation.
    # T5 does not use the concept of 'labels' in the same way, so we don't define them here.
    input_text = f"question: {question_stem} context: " + " ".join([f"choice {chr(65+i)}: {text}" for i, text in enumerate(choices_text)])
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate an answer from the model.
    output_sequences = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=1)

    # Decode the generated text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Here we need a method to match the generated_text to one of the choices.
    # This could be as simple as checking string similarity, or more complex logic.
    # For demonstration, assume we have a function `match_choice_to_generated_text`
    # that returns the index (0-3) of the choice that best matches the generated text.
    predicted_choice_index = match_choice_to_generated_text(generated_text, choices_text)

    # Map the answer key ('A', 'B', 'C', 'D') to an index (0, 1, 2, 3)
    actual_answer_index = ord(answer_key) - ord('A')  # Converts 'A' to 0, 'B' to 1, etc.

    # Compare the predicted index with the actual answer index
    if predicted_choice_index == actual_answer_index:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_questions
print("Accuracy:", accuracy)

TypeError: The current model class (DebertaV2ForMultipleChoice) is not compatible with `.generate()`, as it doesn't have a language model head.

In [ ]:
correct_predictions = 0
for item in data["validation"]:
    # Extract question id, question stem and answer key
    question_id = item['id']
    question_stem = item['question']['stem']
    answer_key = item['answerKey']
    choices = item['question']['choices']

    # Extract choices text
    choices_text = [choice['text'] for choice in choices]

    # Tokenizing inputs as per your specification
    inputs = tokenizer([[question_stem, choices_text[0]], [question_stem, choices_text[1]], [question_stem, choices_text[2]], [question_stem, choices_text[3]]], return_tensors="pt", padding=True)

    # Assuming you have a reason for setting labels like this, even though they're not used for prediction
    labels = torch.tensor(0).unsqueeze(0)

    # Generating outputs with your model as specified
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)

    # Extracting logits and predicting class as specified
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    # Convert the answer key to numeric
    actual_answer_numeric = answer_key_to_numeric[answer_key]

    # Compare the predicted class with the actual answer
    if predicted_class == actual_answer_numeric:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_questions

print(accuracy)